In [36]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from collections import Counter


import warnings

warnings.filterwarnings('ignore')

In [38]:
df_survey_act= pd.read_pickle(f'./dbs/survey_act_weekday.pkl')
df_survey_dist= pd.read_pickle(f'./dbs/intermediate/df_survey_dist.pkl')



In [39]:
# dataf_mddr1116 = pd.read_csv('dbs/travel_survey_se_deso/se_rvu_distribution_mdre1116_deso.csv', delimiter=',',  encoding='latin-1')
#
# dataf_mddr1116= dataf_mddr1116.rename(columns={"uenr":"sub_id",  'aplats':'employee'})
#
# dataf_mddr1116= dataf_mddr1116[["sub_id",  'employee']]
# dataf_mddr1116.drop_duplicates(subset="sub_id", inplace=True)


In [40]:
# create tuple showing all activities of an individual

df_survey_ppl= df_survey_act.groupby('sub_id')['purpose'].apply(tuple).reset_index(name='act_seq')

df_survey_ppl.act_seq.value_counts(normalize=False)

(Home, Work, Home)                                                                         3789
(Home, Other, Home)                                                                        3767
(Home, Home)                                                                               1231
(Home, Other, Home, Other, Home)                                                            976
(Home, Other, Other, Home)                                                                  902
                                                                                           ... 
(Home, Other, Other, Other, Work, Other, Other, Other, Work, Other, Other, Other, Home)       1
(Home, Other, Work, Other, Work, Other, Other, Other, Home, Other, Home)                      1
(Home, Other, Other, Work, Other, Work, Other, Home)                                          1
(Home, Other, Home, Other, Other, Work, Home)                                                 1
(Home, Home, Home, Home, Other, Home, Ho

In [41]:
# find the unique activities from the activity tuple

# def unique_activities(data):
#     act_tuple = data['act_seq']
#     data['act_uniq'] = tuple(Counter(act_tuple).keys())
#     # this will take only unique numbers from the tuple
#     return data
#
# tqdm.pandas()
# df_survey_ppl= df_survey_ppl.apply(lambda row: unique_activities(row) , axis=1 )
#
# df_survey_ppl.act_uniq.value_counts()

### simplify the activity sequences
- If the consecutive activities in the sequences are the same type of anchor activity (Home, Work, School), keep only one of these.
- If there are more than three consecutive Other activities, keep 3 of them.


In [42]:

def act_seq_simplifier(data):
    alist = list(data['act_seq'])
    counter = 0
    for i in range(len(alist) - 1, 0, -1):
        if alist[i] in ['Home', 'Work', 'School']:
            counter = 0
            if alist[i] == alist[i-1]:
                del alist[i]
        elif alist[i] == 'Other':
            counter += 1
            if counter > 3:
                del alist[i]
    data['act_seq_simp'] = tuple(alist)
    return data

tqdm.pandas()
df_survey_ppl = df_survey_ppl.apply(lambda row: act_seq_simplifier(row), axis=1)

df_survey_ppl.act_seq_simp.value_counts(normalize=True)

(Home, Other, Home)                                                  0.217591
(Home, Work, Home)                                                   0.208939
(Home,)                                                              0.077524
(Home, Other, Home, Other, Home)                                     0.054979
(Home, Other, Other, Home)                                           0.051023
                                                                       ...   
(Home, Other, Work, Other, Work, Home, Work, Other, Home)            0.000049
(Home, Other, Other, Other, Work, Other, Work, Other, Work, Home)    0.000049
(Home, Other, Other, School, Other, Home)                            0.000049
(Home, Work, Home, Work, Other, Home, Other, Home)                   0.000049
(Home, Other, School, Home, Other, Home, Other, Home)                0.000049
Name: act_seq_simp, Length: 529, dtype: float64

In [43]:
# remove individuals having only home activity
df_survey_ppl = df_survey_ppl[df_survey_ppl.act_seq_simp != ('Home',)]

#df_survey_ppl.act_seq_simp.value_counts(normalize=True)

## bring the distance information

In [44]:
# bring the distance information

df_survey_ppl = pd.merge(df_survey_ppl, df_survey_dist, on='sub_id', how='left')

df_survey_ppl = df_survey_ppl[df_survey_ppl.avg_dist.notnull()]

df_survey_ppl['commute']= df_survey_ppl['commute'].astype(int)

In [45]:
df_survey_ppl['employee']= df_survey_ppl['employee'].astype(int)


In [46]:
df_survey_ppl['county']= df_survey_ppl['origin_main_deso'].str[:2]

df_survey_ppl['urban_density']= df_survey_ppl['origin_main_deso'].str[4]

In [47]:
#df_survey_ppl.act_seq_simp.value_counts(normalize=True)

In [48]:
print('share of people participating work activity: ', np.round(df_survey_ppl.commute.sum()/len(df_survey_ppl),2))

share of people participating work activity:  0.52


In [49]:
#df_survey_ppl = pd.merge(df_survey_ppl, dataf_mddr1116, on='sub_id', how='left')

# assigin all people participating work activity as employee (there are 13 ppl not employee but participate work)
df_survey_ppl['employee'][(df_survey_ppl.employee==0)&(df_survey_ppl.commute==1)]=1


print('share of employee: ', np.round(df_survey_ppl.employee.sum()/len(df_survey_ppl),2))



share of employee:  0.7


In [50]:
df_survey_ppl = df_survey_ppl[['sub_id', 'origin_main_deso', 'employee',
       'commute', 'act_seq', 'act_seq_simp', 'avg_dist', 'com_distance', 'county', 'urban_density']]




In [51]:
df_survey_ppl['modified'] = 0

df_survey_ppl['modified'][df_survey_ppl['act_seq']!=df_survey_ppl['act_seq_simp']]=1

In [52]:
print('the share of the simplified activity sequence: ',np.round((df_survey_ppl['modified'].sum())/ len(df_survey_ppl),2))

the share of the simplified activity sequence:  0.15


In [53]:
df_survey_ppl.to_pickle(r'.\dbs\intermediate\df_survey_twins.pkl')